# MT-Bench

## Load The Environment

Make sure that you have created a .env file containing the OPENAI_API_KEY:

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv("OPENAI_API_KEY") is not None:
    print("OpenAI API key present")
else:
    print("OpenAI API key not present")

OpenAI API key present


## Load The Dataset

The `limin_bench` package provides native functionality to load the MT-bench dataset. You can either call `load_mt_bench()` to get the entire dataset or get only specific categories by passing the list of strings containing the categories. For simplicity, we will only select the data from the `writing` category in this notebook:

In [9]:
from limin_bench import load_mt_bench
dataset = load_mt_bench(["writing"])

The MT-bench dataset is a `PregeneratedMultiTurnDataset` meaning that every dataset row is a list of strings containing the pregenerated multi-turn user messages:

In [10]:
type(dataset)

limin_bench.base.PregeneratedMultiTurnDataset

Every dataset behaves like a regular iterable, so you can its length and index into it:

In [11]:
len(dataset)

10

In [12]:
print(dataset[0])

['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.', 'Rewrite your previous response. Start every sentence with the letter A.']


You can also pretty-print a dataset as a markdown table. This will print the rows as well as the individual turns:

In [14]:
print(dataset.to_markdown_table(max_column_length=100))

| Row | Turn | Message                                                                                              |
|-----|------|------------------------------------------------------------------------------------------------------|
| 0   | 1    | Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experie... |
|     | 2    | Rewrite your previous response. Start every sentence with the letter A.                              |
| 1   | 1    | Draft a professional email seeking your supervisor’s feedback on the ‘Quarterly Financial Report’... |
|     | 2    | Take a moment to evaluate and critique your own response.                                            |
| 2   | 1    | Imagine you are writing a blog post comparing two popular smartphone models. Develop an outline f... |
|     | 2    | Take your previous response and rephrase it as a limerick.                                           |
| 3   | 1    | Write a persuasive email to convince your

## Creating a Model Run

Let's configure the model that we want to evaluate:

In [15]:
from limin import ModelConfiguration

assistant_model_configuration = ModelConfiguration(model="gpt-4o", temperature=1.0)
assistant_system_prompt = "You are a helpful assistant."

To generate a model run from a `PregeneratedMultiTurnDataset` you can use the `generate_multi_turn_model_run_from_pregenerated_dataset` function:

In [17]:
from limin_bench import generate_multi_turn_model_run_from_pregenerated_dataset

model_run = await generate_multi_turn_model_run_from_pregenerated_dataset(
    dataset=dataset,
    assistant_system_prompt=assistant_system_prompt,
    assistant_model_configuration=assistant_model_configuration
)

Generating model run rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:59<00:00,  5.98s/it]


Just like the dataset, the model run is an iterable - you can get its length and index into it. Additionally, you can pretty-print it as a markdown table:

In [18]:
print(model_run.to_markdown_table(max_column_length=50))

| Row | Turn | Role      | Message                                            |
|-----|------|-----------|----------------------------------------------------|
| 0   | 0    | system    | You are a helpful assistant.                       |
|     | 1    | user      | Compose an engaging travel blog post about a re... |
|     |      | assistant | Title: Aloha Adventure: Embarking on a Cultural... |
|     | 2    | user      | Rewrite your previous response. Start every sen... |
|     |      | assistant | Aloha, dear readers! Adventure beckoned, and my... |
| 1   | 0    | system    | You are a helpful assistant.                       |
|     | 1    | user      | Draft a professional email seeking your supervi... |
|     |      | assistant | Subject: Request for Feedback on Quarterly Fina... |
|     | 2    | user      | Take a moment to evaluate and critique your own... |
|     |      | assistant | Thank you for the opportunity to evaluate my re... |
| 2   | 0    | system    | You are a hel

## Creating an Evaluation Run

Let's now configure the Likert judge that we use to evaluate the model (or, technically, the model run). We will lift the system prompt straight from the MT-Bench paper except that we will use the `structured` argument to parse the response:

In [20]:
from limin_bench import LikertJudge

In [21]:
judge_system_prompt = """
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below.
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response.
You evaluation should focus on the assistant's answer to the second user question.
Begin your evaluation by providing a short explanation.
Be as objective as possible.
After providing your explanation, you must rate the response on a scale of 1 to 10.
"""

likert_judge = LikertJudge(
    model_configuration=ModelConfiguration(model="gpt-4o", temperature=0.4),
    system_prompt=judge_system_prompt
)

Now we can execute the evaluation run. Even though have a pregenerated multi-turn dataset executing an evaluation run works the same as with a regular single-turn dataset. This is the beauty of `limin-bench` - evaluation runs don't care about the initial structure of your dataset, they only care about how the model run looks like. And the model run always looks the same - it's a list of conversations. We will set `n_stability_runs=2` - this means that for every model run row we will have the judge evaluate the row `2` times and then check whether the values match. It is always recommended to set `n_stability_runs > 1` in order to verify that your judge isn't just producing arbitrary guesses:

In [23]:
from limin_bench import generate_evaluation_run_likert

evaluation_run = await generate_evaluation_run_likert(
    model_run=model_run,
    likert_judge=likert_judge,
    n_stability_runs=2,
    structured=True
)

Generating likert evaluation run: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.17s/it]


Once again, we can pretty print the result as a markdown table:

In [25]:
print(evaluation_run.to_markdown_table(max_column_length=30))

| Row | Turn | Role      | Message                        | Explanation                    | Score | Instability |
|-----|------|-----------|--------------------------------|--------------------------------|-------|-------------|
| 0   | 0    | system    | You are a helpful assistant.   |                                |       |             |
|     | 1    | user      | Compose an engaging travel ... |                                |       |             |
|     |      | assistant | Title: Aloha Adventure: Emb... |                                |       |             |
|     | 2    | user      | Rewrite your previous respo... |                                |       |             |
|     |      | assistant | Aloha, dear readers! Advent... | The AI assistant successful... | 8.0   | 0.0         |
| 1   | 0    | system    | You are a helpful assistant.   |                                |       |             |
|     | 1    | user      | Draft a professional email ... |                     

We can see a few rows with an instability value greater than `0`. This means that for these rows the judge produced different values between runs. We can check such a row in more detail:

In [27]:
unstable_row = evaluation_run[6]

Here are the full results, including the judge response (explanation) and the final value:

In [32]:
unstable_row.results

[LikertEvaluationRunRowResult(judge_response="The assistant effectively responded to the user's request to rewrite the story using four-word sentences. The response maintained the essence of the original story while adhering to the constraint of using only four-word sentences. This approach resulted in a concise and focused narrative that still conveyed the key elements of the plot. The assistant demonstrated creativity by successfully adapting the story to the new format without losing coherence or depth. Overall, the response was relevant, accurate, and showed a good level of detail given the constraints.", value=9, explanation="The assistant effectively responded to the user's request to rewrite the story using four-word sentences. The response maintained the essence of the original story while adhering to the constraint of using only four-word sentences. This approach resulted in a concise and focused narrative that still conveyed the key elements of the plot. The assistant demonst

We can calculate the final value of the evaluation run row by taking either the mean (default), min or max result between the values:

In [31]:
unstable_row.value()

8.5

In [33]:
unstable_row.value("mean")

8.5

In [34]:
unstable_row.value("max")

9

In [35]:
unstable_row.value("min")

8

Additionally, we can get the instability of a single evaluation row - the higher the instability, the more different the judge responses were between evaluations, the more unusable the evaluation is. The instability is calculated as the standard deviation of the values.

In [36]:
unstable_row.instability

0.7071067811865476

Finally, we can calculate the total average value and the total instability of the evaluation run:

In [37]:
evaluation_run.avg

8.55

In [39]:
evaluation_run.instability()

0.21213203435596428

We have an average score of `8.5 / 10` and an instability of `0.21` indicating that the evaluation is good and useful. Of course, a low instability is not a definitive indicator that you have a usable evaluation - maybe you just wrote a prompt that is too easy. Nevertheless, it is still useful to get an overall sense of how well you structured your evaluation - a high instability is a definitive indicator of an unusable evaluation.